<a href="https://colab.research.google.com/github/yacinmansour161-sudo/Malaria-Detection/blob/main/last_chance_malariya_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import torch
torch.cuda.is_available()

True

In [26]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.9.0+cu126
0.24.0+cu126


In [27]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{\r\n  "username": "yassou IA",\r\n  "key": "KGAT_ad8ac65fdf179a0f07e278e4c62e83a7"\r\n}\r\n'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria
!unzip cell-images-for-detecting-malaria.zip


Dataset URL: https://www.kaggle.com/datasets/iarunava/cell-images-for-detecting-malaria
License(s): unknown
cell-images-for-detecting-malaria.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  cell-images-for-detecting-malaria.zip
replace cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_162.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import shutil
import os

duplicate_path = "cell_images/cell_images"

if os.path.exists(duplicate_path):
    shutil.rmtree(duplicate_path)
    print("✅ Dossier dupliqué supprimé")
else:
    print("ℹ️ Aucun dossier dupliqué trouvé")


In [ ]:
os.listdir("cell_images")


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
import os

os.listdir("/content")
os.listdir("/content/cell_images")


In [ ]:
import os
import shutil
import random

SOURCE_DIR = "cell_images"
TARGET_DIR = "data"

SPLITS = {
    "train": 0.7,
    "val": 0.15,
    "test": 0.15
}

CLASSES = ["Parasitized", "Uninfected"]

random.seed(42)

for cls in CLASSES:
    images = os.listdir(os.path.join(SOURCE_DIR, cls))
    random.shuffle(images)

    n = len(images)
    train_end = int(SPLITS["train"] * n)
    val_end = train_end + int(SPLITS["val"] * n)

    split_images = {
        "train": images[:train_end],
        "val": images[train_end:val_end],
        "test": images[val_end:]
    }

    for split, imgs in split_images.items():
        split_dir = os.path.join(TARGET_DIR, split, cls)
        os.makedirs(split_dir, exist_ok=True)

        for img in imgs:
            shutil.copy(
                os.path.join(SOURCE_DIR, cls, img),
                os.path.join(split_dir, img)
            )

print("✅ Split train/val/test terminé")


In [ ]:
for split in ["train", "val", "test"]:
    print(f"\n{split.upper()}")
    for cls in CLASSES:
        path = f"data/{split}/{cls}"
        print(cls, ":", len(os.listdir(path)))


In [ ]:
train_dataset = datasets.ImageFolder("data/train", transform=train_transform)
val_dataset   = datasets.ImageFolder("data/val", transform=val_test_transform)
test_dataset  = datasets.ImageFolder("data/test", transform=val_test_transform)


In [ ]:
BATCH_SIZE = 32

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [ ]:
images, labels = next(iter(train_loader))

plt.figure(figsize=(10,5))
for i in range(6):
    plt.subplot(2,3,i+1)
    img = images[i].permute(1,2,0)
    img = img * 0.225 + 0.45  # dé-normalisation approx
    plt.imshow(img.clamp(0,1))
    plt.title(train_dataset.classes[labels[i]])
    plt.axis("off")
plt.show()


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class MalariaCNN(nn.Module):
    def __init__(self):
        super(MalariaCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, 2)  # 2 classes

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))

        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)

        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MalariaCNN().to(device)
print(model)


In [ ]:
dummy_input = torch.randn(1, 3, 224, 224).to(device)
output = model(dummy_input)
print(output.shape)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
from tqdm import tqdm

NUM_EPOCHS = 10
best_val_acc = 0.0

for epoch in range(NUM_EPOCHS):
    # --- Entraînement ---
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels)
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = running_corrects.double() / total
    print(f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f}")

    # --- Validation ---
    model.eval()
    val_corrects = 0
    val_total = 0


    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels)
            val_total += labels.size(0)

    val_acc = val_corrects.double() / val_total
    print(f"Val Acc: {val_acc:.4f}")

    # Sauvegarde du meilleur modèle
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_malaria_cnn.pth")
        print("💾 Nouveau meilleur modèle sauvegardé !")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Charger le meilleur modèle
model.load_state_dict(torch.load("best_malaria_cnn.pth"))
model.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("Matrice de confusion :")
print(confusion_matrix(y_true, y_pred))
print("\nClassification report :")
print(classification_report(y_true, y_pred, target_names=test_dataset.classes))
